### Import Libraries

In [1]:
import pandas as pd
import numpy as np

### Step 1. Data Loading and Initial Inspection

In [2]:
# Load the dataset
df = pd.read_excel('Onyx Data - DataDNA Dataset Challenge - Business Financial Dataset - August 2024.xlsx')

# Check data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     580 non-null    int64         
 1   Month - name             580 non-null    object        
 2   Month -sequence          580 non-null    int64         
 3   Date                     580 non-null    datetime64[ns]
 4   Business Line            580 non-null    object        
 5   Amount, $                580 non-null    int64         
 6   Expense subgroup         468 non-null    object        
 7   Revenue / Expense Group  580 non-null    object        
 8   Revenue or expense       580 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 40.9+ KB


In [3]:
df.head()

,Year,Month - name,Month -sequence,Date,Business Line,"Amount, $",Expense subgroup,Revenue / Expense Group,Revenue or expense
0,2023,January,1,2023-01-31,Nutrition and Food Supplements,153000,NaN,Sales,Revenue
1,2023,January,1,2023-01-31,Nutrition and Food Supplements,27000,NaN,Consulting and professional services,Revenue
2,2023,January,1,2023-01-31,Nutrition and Food Supplements,6000,NaN,Other income,Revenue
3,2023,January,1,2023-01-31,Nutrition and Food Supplements,-15000,Rent,Opex,Expense
4,2023,January,1,2023-01-31,Nutrition and Food Supplements,-9000,Equipment,Opex,Expense


### Step 1. Data Cleaning and Preprocessing

In [4]:
df.isnull().sum()

Year                         0
Month - name                 0
Month -sequence              0
Date                         0
Business Line                0
Amount, $                    0
Expense subgroup           112
Revenue / Expense Group      0
Revenue or expense           0
dtype: int64

In [5]:
# Define the mappings for quarters and half-years
quarter_mapping = {
    'January': 'Q1', 'February': 'Q1', 'March': 'Q1',
    'April': 'Q2', 'May': 'Q2', 'June': 'Q2',
    'July': 'Q3', 'August': 'Q3', 'September': 'Q3',
    'October': 'Q4', 'November': 'Q4', 'December': 'Q4'
}
 
quarter_sequence_mapping = {
    'Q1': 1, 'Q2': 2, 'Q3': 3, 'Q4': 4
}
 
half_year_mapping = {
    'Q1': 'H1', 'Q2': 'H1',
    'Q3': 'H2', 'Q4': 'H2'
}
 
half_year_sequence_mapping = {
    'H1': 1, 'H2': 2
}
 
# Apply the mappings to create new columns
df['Quarter'] = df['Month - name'].map(quarter_mapping)
df['Quarter - Sequence'] = df['Quarter'].map(quarter_sequence_mapping)
df['Half Year'] = df['Quarter'].map(half_year_mapping)
df['Half Year - Sequence'] = df['Half Year'].map(half_year_sequence_mapping)
 
# Clean data
df['Expense subgroup'] = df['Expense subgroup'].fillna('Other')
df['Amount, $'] = df['Amount, $'].fillna(0)

 
# Adjust the day of the week mapping so that Monday starts with 1
df['Day of Week'] = df['Date'].dt.dayofweek + 1
 
# Map the day of the week to their names
day_of_week_map = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday', 6: 'Saturday', 7: 'Sunday'}
df['Day of Week - Name'] = df['Day of Week'].map(day_of_week_map)
 
# Create the Is Weekend indicator
df['Is Weekend'] = df['Day of Week'].isin([6, 7])

In [6]:
# Create Line Item ID
df["Line Item ID"] = df["Revenue or expense"] + " - " + df["Revenue / Expense Group"] + " - " + df["Expense subgroup"]


# Replace Column Name
df = df.rename(columns={'Amount, $':"Amount"})
df["Amount - Copy"] = df["Amount"].copy()
df["Amount"] = df["Amount"].abs()

In [7]:
# Group the DataFrame by 'Date' and 'Line Item ID' and sum the 'Amount'
grouped_df = df.groupby(['Date', 'Line Item ID', 'Expense subgroup','Revenue / Expense Group','Revenue or expense'])['Amount'].sum().reset_index()

# Calculate Sales
sales = grouped_df[grouped_df['Revenue or expense'] == 'Revenue'].groupby('Date')['Amount'].sum().reset_index()
sales['Line Item ID'] = 'Sales'

# Calculate Expense - COGS
expense_cogs = grouped_df[grouped_df['Revenue / Expense Group'] == 'COGS'].groupby('Date')['Amount'].sum().reset_index()
expense_cogs['Line Item ID'] = 'Expense - COGS'

# Calculate Gross Profit (Sales - Expense - COGS)
gross_profit = pd.merge(sales, expense_cogs, on='Date', how='left', suffixes=('_Sales', '_COGS'))
gross_profit['Amount'] = gross_profit['Amount_Sales'] - gross_profit['Amount_COGS']
gross_profit['Line Item ID'] = 'Gross Profit'
gross_profit = gross_profit[['Date', 'Amount', 'Line Item ID']]

# Calculate Operating Expenses (Opex)
operating_expenses = grouped_df[grouped_df['Revenue / Expense Group'] == 'Opex'].groupby('Date')['Amount'].sum().reset_index()
operating_expenses['Line Item ID'] = 'Operating Expenses'

# Calculate EBIT (Gross Profit - Operating Expenses)
ebit = pd.merge(gross_profit, operating_expenses, on='Date', how='left', suffixes=('_GrossProfit', '_Opex'))
ebit['Amount'] = ebit['Amount_GrossProfit'] - ebit['Amount_Opex']
ebit['Line Item ID'] = 'EBIT'
ebit = ebit[['Date', 'Amount', 'Line Item ID']]

# Calculate Net Profit (EBIT - Interest and Tax)
interest_tax = grouped_df[grouped_df['Revenue / Expense Group'] == 'Interest and tax'].groupby('Date')['Amount'].sum().reset_index()
interest_tax['Line Item ID'] = 'Interest and Tax'
net_profit = pd.merge(ebit, interest_tax, on='Date', how='left', suffixes=('_EBIT', '_InterestTax'))
net_profit['Amount'] = net_profit['Amount_EBIT'] - net_profit['Amount_InterestTax']
net_profit['Line Item ID'] = 'Net Profit'
net_profit = net_profit[['Date', 'Amount', 'Line Item ID']]

# Combine all calculations into the original grouped DataFrame
new_rows = pd.concat([sales, expense_cogs, gross_profit, operating_expenses, ebit, net_profit])
final_df = pd.concat([grouped_df, new_rows]).sort_values(by=['Date', 'Line Item ID']).reset_index(drop=True)
final_df

,Date,Line Item ID,Expense subgroup,Revenue / Expense Group,Revenue or expense,Amount
0,2023-01-31,EBIT,NaN,NaN,NaN,807500
1,2023-01-31,Expense - COGS,NaN,NaN,NaN,662600
2,2023-01-31,Expense - COGS - Labor,Labor,COGS,Expense,345000
3,2023-01-31,Expense - COGS - Materials,Materials,COGS,Expense,100000
4,2023-01-31,Expense - COGS - Other,Other,COGS,Expense,21700
...,...,...,...,...,...,...
259,2023-12-31,Operating Expenses,NaN,NaN,NaN,536800
260,2023-12-31,Revenue - Consulting and professional services...,Other,Consulting and professional services,Revenue,270900
261,2023-12-31,Revenue - Other income - Other,Other,Other income,Revenue,96000
262,2023-12-31,Revenue - Sales - Other,Other,Sales,Revenue,1363275


In [8]:
# Check for missing values
missing_values = df.isnull().sum()
print("Missing Values:\n", missing_values)
 
# Check for duplicated rows
duplicated_rows = df.duplicated().sum()
if duplicated_rows > 0:
    df = df.drop_duplicates()
 
# Convert 'Date' column to datetime if not already
if df['Date'].dtype != 'datetime64[ns]':
    df['Date'] = pd.to_datetime(df['Date'])

Missing Values:
 Year                       0
Month - name               0
Month -sequence            0
Date                       0
Business Line              0
Amount                     0
Expense subgroup           0
Revenue / Expense Group    0
Revenue or expense         0
Quarter                    0
Quarter - Sequence         0
Half Year                  0
Half Year - Sequence       0
Day of Week                0
Day of Week - Name         0
Is Weekend                 0
Line Item ID               0
Amount - Copy              0
dtype: int64


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 580 entries, 0 to 579
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Year                     580 non-null    int64         
 1   Month - name             580 non-null    object        
 2   Month -sequence          580 non-null    int64         
 3   Date                     580 non-null    datetime64[ns]
 4   Business Line            580 non-null    object        
 5   Amount                   580 non-null    int64         
 6   Expense subgroup         580 non-null    object        
 7   Revenue / Expense Group  580 non-null    object        
 8   Revenue or expense       580 non-null    object        
 9   Quarter                  580 non-null    object        
 10  Quarter - Sequence       580 non-null    int64         
 11  Half Year                580 non-null    object        
 12  Half Year - Sequence     580 non-nul

In [10]:
 
# Save the cleaned and preprocessed DataFrame to a new file
df.to_excel('Cleaned_Onyx Data - DataDNA Dataset Challenge - Business Financial Dataset - August 2024.xlsx', index=False)
final_df.to_excel('final_df.xlsx', index=False)